# SCIDONI : Prediction of the isotopic inventory in a nuclear reactor core

*Benjamin Dechenaux & Jean-Baptiste Clavel (IRSN, PSN-EXP/SNC/LN)*

## Introduction 

This project aims at the prediction of the time evolving composition of the nuclear fuel inside a reactor.

Building a model capable of predicting such inventory is usually done through computer expensive "Monte-Carlo" calculations and the goal here would be to build a fast and relatively precise model to use in an emergency situation such as the occurence of a severe accident.


In [19]:
# Required for running the notebook 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle

%matplotlib inline

## About the data 

We propose to track and predict the temporal evolution of 26 isotopes that, for confidentiality reasons, were renammed in the data and are identifed by letters of the alphabet.

So we'll be interested in the evolution of the content in isotopes "A", "B", ..., "Z" inside a reactor with time. The isotope content at each time is expressed in some kind of density unity.  

### Modelling the irradiation in a nuclear reactor

To model the irradiation conditions of the nuclear material, we suppose a simplified scheme for the temporal evolution.

The fuel is put inside the reactor for a grand total of __1825 days__. 

This total period is subdivided into 5 different irradiation __cycles__ where each cycle correspond to a 300 days period with a certain configuration of the reactor, charaterized by a single parameter *pi* (with i running from 1 -> 5).  

Between 2 cycles, a period of 65 days was added and correspond to say, a maintenance period for the reactor, where the fuel is not irradiated (which note, doesn't mean the isotopes don't evolve). During these intercycle period, we don't track the evolution of the isotopes content.

To summarize, the nuclear fuel is put inside of a reactor for a total of 1825 days with the following history :

* The fuel is irradiated for 300 days (from T = 0 to T = 300) with parameter *p1*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 365 to T = 665) with parameter *p2*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 730 to T = 1030) with parameter *p3*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 1095 to T = 1395) with parameter *p4*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 1460 to T = 1760) with parameter *p5*
* The fuel is put to rest for 65 days


For the sake of the exercice, the composition of each isotope will be tracked on a __20 days__ time interval basis, except for the periods where the fuel is put to rest, where the interval is 65 days (i.e. the evolution of fuel composition is not being tracked, which doesn't mean it doesn't evolve...) . 


So in the end, for a given set of input data (which consist of the inital composition of isotopes A -> H and the 5 parameters p1,...,p5), the result is a time series of length 81 (initial composition + 80 timesteps).


### Description of the available data

A set of different input data were generated, varying the initial composition of the material in terms of isotopes __A to H__ (all the other isotopes, I -> Z, are *always* 0. at the beginning) and the parameters p1 to p5 (to generate them we used the Latin Hypercube method). 

A total of __1120__ different input data sets were generated and used to perform as many simulation of the evolution of the material. 

In each case, a CSV output was generated and consist of the time series representing the temporal composition of each isotope with time.  


The 1120 simulations have been split into two *training* and *testing* datasets.

* The *training* dataset is composed of 920 simulations and are accessible in CSV format undert the __train__ folder
* The *testing* dataset is composed of 200 simulations and are accessible in CSV format undert the __test__ folder


### Loading the data 

To ease the reading of both the training and testing datasets, serialized *pandas* dataframes have been pre-prepared and can be loaded using *pickle*

In [3]:
dtrain = pickle.load( open("data/train_data_python3.pickle", "rb") )
dtest = pickle.load( open("data/test_data_python3.pickle", "rb") )

In these dataframes, data have been concatenated one on top of the other. 

To separate from one file to the other, use the index of the dataframes or the "times" column.

In [20]:
dtrain.head(5)

,times,A,B,C,D,E,F,G,H,I,...,V,W,X,Y,Z,p1,p2,p3,p4,p5
0,0.0,0.173951,4.219378,0.088625,2.014683,0.677343,0.079891,0.054258,0.164877,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.04842,0.029876,0.043997,0.03811,0.042866
1,20.0,0.172232,4.214907,0.088131,1.995882,0.678855,0.083688,0.054383,0.162316,0.000005,...,0.000038,0.000005,0.000160,0.000158,0.000362,0.04842,0.029876,0.043997,0.03811,0.042866
2,40.0,0.170516,4.210405,0.087780,1.977858,0.680277,0.087465,0.054536,0.159785,0.000031,...,0.000084,0.000022,0.000540,0.000316,0.000485,0.04842,0.029876,0.043997,0.03811,0.042866
3,60.0,0.168814,4.205899,0.087564,1.959920,0.681660,0.091179,0.054690,0.157300,0.000092,...,0.000132,0.000050,0.000999,0.000474,0.000526,0.04842,0.029876,0.043997,0.03811,0.042866
4,80.0,0.167123,4.201368,0.087465,1.942086,0.682998,0.094833,0.054855,0.154862,0.000190,...,0.000182,0.000088,0.001487,0.000631,0.000540,0.04842,0.029876,0.043997,0.03811,0.042866


Here is an example of how to get the initial "0 days" timesteps for all of the training sample. 
Simply do :

In [8]:
dtrain.loc[0].shape  # equivalent to dtrain.loc[dtrain["times"] == 0.]

(920, 32)

As said earlier, the data consist of 920 evolution data with 32 columns (one is "times", some are for "p1" to "p5" and the others are for the compositions "A" -> "Z").

Wa can have a look at the initial parameters that were given to the simulation for the fisrt test case :

In [21]:
dtrain.iloc[0]

times    0.000000
A        0.173951
B        4.219378
C        0.088625
D        2.014683
E        0.677343
F        0.079891
G        0.054258
H        0.164877
I        0.000000
J        0.000000
K        0.000000
L        0.000000
M        0.000000
N        0.000000
O        0.000000
P        0.000000
Q        0.000000
R        0.000000
S        0.000000
T        0.000000
U        0.000000
V        0.000000
W        0.000000
X        0.000000
Y        0.000000
Z        0.000000
p1       0.048420
p2       0.029876
p3       0.043997
p4       0.038110
p5       0.042866
Name: 0, dtype: float64

As adverstised, the initial compositions for isotopes "I" -> "Z" are zero. This is not the case for the last timestep :

In [22]:
dtrain.iloc[[0,80]]

,times,A,B,C,D,E,F,G,H,I,...,V,W,X,Y,Z,p1,p2,p3,p4,p5
0,0.0,0.173951,4.219378,0.088625,2.014683,0.677343,0.079891,0.054258,0.164877,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.04842,0.029876,0.043997,0.03811,0.042866
80,1825.0,0.085919,3.912349,0.119914,1.042389,0.682391,0.222099,0.076751,0.073000,0.01985,...,0.004407,0.014348,0.028545,0.009016,0.000014,0.04842,0.029876,0.043997,0.03811,0.042866


To get all of the timesteps, one can use the barbaric but efficient :

In [25]:
timesteps = sorted(list(set(dtrain["times"])))
timesteps

[0.0,
 20.0,
 40.0,
 60.0,
 80.0,
 100.0,
 120.0,
 140.0,
 160.0,
 180.0,
 200.0,
 220.0,
 240.0,
 260.0,
 280.0,
 300.0,
 365.0,
 385.0,
 405.0,
 425.0,
 445.0,
 465.0,
 485.0,
 505.0,
 525.0,
 545.0,
 565.0,
 585.0,
 605.0,
 625.0,
 645.0,
 665.0,
 730.0,
 750.0,
 770.0,
 790.0,
 810.0,
 830.0,
 850.0,
 870.0,
 890.0,
 910.0,
 930.0,
 950.0,
 970.0,
 990.0,
 1010.0,
 1030.0,
 1095.0,
 1115.0,
 1135.0,
 1155.0,
 1175.0,
 1195.0,
 1215.0,
 1235.0,
 1255.0,
 1275.0,
 1295.0,
 1315.0,
 1335.0,
 1355.0,
 1375.0,
 1395.0,
 1460.0,
 1480.0,
 1500.0,
 1520.0,
 1540.0,
 1560.0,
 1580.0,
 1600.0,
 1620.0,
 1640.0,
 1660.0,
 1680.0,
 1700.0,
 1720.0,
 1740.0,
 1760.0,
 1825.0]

## Our goal 

The goal we have in mind is to predict the __whole time evolution__ from only the __initial compositions at T = 0 days__.

Enjoy !
